In [1]:
import torch
import torchani
from torchani.datasets import ANIDataset

import ase
from ase.visualize import view
from ase.optimize import BFGS
from ase import io

import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
ds_path = Path('/home/nick/First_DSs/ANI-1x-first-conformers.h5')
ds = ANIDataset(locations=ds_path, names='1x first')
device = torch.device('cpu')
ani2x = torchani.models.ANI2x(periodic_table_index=True).to(device)
calculator = ani2x.ase()

/home/nick/TorchANI_sandbox/torchani/models.py:103: UserWarning: The default is now to accept atomic numbers as indexes, do not set periodic_table_index=True. if you need to accept raw indices set periodic_table_index=False
  warnings.warn("The default is now to accept atomic numbers as indexes,"


In [3]:
c5h12 = io.read('c5h12.xyz')

molecule = c5h12
molecule.calc = calculator
dyn = BFGS(molecule)
dyn.run(fmax=0.05)
view(molecule,viewer='ngl')

      Step     Time          Energy         fmax
BFGS:    0 09:01:38    -5380.198751        0.0880
BFGS:    1 09:01:39    -5380.199212        0.0230


In [4]:
c2h8n6o2 = ds['C2H8N6O2']

numbers=c2h8n6o2['species'][0]
positions=c2h8n6o2['coordinates'][0]

molecule = ase.Atoms(numbers=numbers, positions=positions)
molecule.calc = calculator
dyn = BFGS(molecule)
dyn.run(fmax=0.1)
view(molecule,viewer='ngl')

      Step     Time          Energy         fmax
BFGS:    0 09:01:39   -15229.478990        7.4088
BFGS:    1 09:01:39   -15230.829487        6.6516
BFGS:    2 09:01:39   -15229.739153       14.1237
BFGS:    3 09:01:39   -15231.299211        3.5097
BFGS:    4 09:01:39   -15231.418999        2.8005
BFGS:    5 09:01:39   -15231.596511        1.3021
BFGS:    6 09:01:39   -15231.667273        0.9293
BFGS:    7 09:01:39   -15231.736230        1.5529
BFGS:    8 09:01:39   -15231.800476        1.4086
BFGS:    9 09:01:39   -15231.854206        0.9422
BFGS:   10 09:01:39   -15231.886514        0.7653
BFGS:   11 09:01:39   -15231.918168        0.6847
BFGS:   12 09:01:39   -15231.942175        0.5291
BFGS:   13 09:01:39   -15231.955255        0.4243
BFGS:   14 09:01:39   -15231.968978        0.5007
BFGS:   15 09:01:39   -15231.984198        0.5307
BFGS:   16 09:01:39   -15231.998313        0.3919
BFGS:   17 09:01:39   -15232.016887        0.3769
BFGS:   18 09:01:39   -15232.030635        0.4703
B

In [5]:
#ase.io.write('c2h8n6o2.xyz',molecule)

In [6]:
from pathlib import Path
from torchani.datasets import ANIDataset

path1x = Path('/home/nick/Datasets/ani1x/ANI-1x-wB97X-631Gd.h5')
ds_1x = ANIDataset(locations=path1x,names='1x')

In [7]:
print(len(ds_1x['C4H4']['energies']))
print((ds_1x['CH4']['energies'].mean()))
print((ds_1x['CH4']['energies'].min()))

1781
tensor(-40.4774, dtype=torch.float64)
tensor(-40.4996, dtype=torch.float64)


In [8]:
diff = ds_1x['CH4']['energies'].min() - ds_1x['CH4']['energies'].mean()

In [9]:
from torchani.units import hartree2kcalmol
torch.set_printoptions(10)
hartree2kcalmol(diff)

tensor(-13.9399583791, dtype=torch.float64)

In [10]:
path2x = Path('/home/nick/Datasets/ani2x/ALL_COMBINED.h5')
ds_2x = ANIDataset(locations=path2x, names='2x')
print(len(ds_2x))

15992


In [11]:
a = 0
for i in ds_2x:
    a += len(ds_2x[str(i)]['coordinates'])
print(a)

9651712


In [12]:
species = ds_2x['CH4']['species'][0][None]
print(species)

tensor([[6, 1, 1, 1, 1]])


In [13]:
numbers = ds_2x['CH4']['species'][0].numpy()

In [14]:
ani2x = torchani.models.ANI2x().double()

/home/nick/TorchANI_sandbox/torchani/models.py:103: UserWarning: The default is now to accept atomic numbers as indexes, do not set periodic_table_index=True. if you need to accept raw indices set periodic_table_index=False
  warnings.warn("The default is now to accept atomic numbers as indexes,"


In [15]:
ch4_energies = []

for i in ds_2x['CH4']['coordinates']:
    positions = i.numpy()
    molecule = ase.Atoms(numbers=numbers, positions=positions)
    molecule.calc = calculator
    dyn = BFGS(molecule)
    dyn.run(fmax=0.05)
    new_coord = torch.tensor(molecule.positions)[None]
    ani_inp = (species, new_coord)    
    ch4_energies.append(ani2x(ani_inp).energies.item())
print(ch4_energies)

      Step     Time          Energy         fmax
BFGS:    0 09:02:30    -1101.652213        5.4868
BFGS:    1 09:02:30    -1101.950568        1.6234
BFGS:    2 09:02:30    -1101.991349        0.8651
BFGS:    3 09:02:30    -1102.020865        0.5139
BFGS:    4 09:02:30    -1102.038562        0.4610
BFGS:    5 09:02:30    -1102.044180        0.1599
BFGS:    6 09:02:30    -1102.045011        0.0669
BFGS:    7 09:02:30    -1102.045153        0.0287
      Step     Time          Energy         fmax
BFGS:    0 09:02:30    -1101.833738        2.1826
BFGS:    1 09:02:30    -1101.943788        0.9112
BFGS:    2 09:02:30    -1101.988297        0.8374
BFGS:    3 09:02:30    -1102.028790        0.4803
BFGS:    4 09:02:30    -1102.039293        0.3093
BFGS:    5 09:02:30    -1102.044380        0.1138
BFGS:    6 09:02:30    -1102.044935        0.0527
BFGS:    7 09:02:30    -1102.045077        0.0387
      Step     Time          Energy         fmax
BFGS:    0 09:02:30    -1101.922680        2.0000
BFG

BFGS:    6 09:02:32    -1102.044099        0.0905
BFGS:    7 09:02:32    -1102.044562        0.0859
BFGS:    8 09:02:32    -1102.045048        0.0626
BFGS:    9 09:02:32    -1102.045238        0.0271
      Step     Time          Energy         fmax
BFGS:    0 09:02:32    -1102.040491        0.3771
BFGS:    1 09:02:32    -1102.042658        0.1467
BFGS:    2 09:02:32    -1102.043819        0.1049
BFGS:    3 09:02:32    -1102.045172        0.0525
BFGS:    4 09:02:32    -1102.045242        0.0221
      Step     Time          Energy         fmax
BFGS:    0 09:02:33    -1101.212343        5.6751
BFGS:    1 09:02:33    -1101.655751        1.7610
BFGS:    2 09:02:33    -1101.830030        1.4948
BFGS:    3 09:02:33    -1102.005364        0.8693
BFGS:    4 09:02:33    -1102.032506        0.4670
BFGS:    5 09:02:33    -1102.042944        0.1909
BFGS:    6 09:02:33    -1102.044037        0.0972
BFGS:    7 09:02:33    -1102.044576        0.0735
BFGS:    8 09:02:33    -1102.045038        0.0616
BF

BFGS:   10 09:02:35    -1102.045255        0.0244
      Step     Time          Energy         fmax
BFGS:    0 09:02:35    -1101.463570        6.0262
BFGS:    1 09:02:35    -1101.757103        2.1230
BFGS:    2 09:02:35    -1101.869529        1.1477
BFGS:    3 09:02:35    -1101.995028        0.9616
BFGS:    4 09:02:35    -1102.028493        0.6635
BFGS:    5 09:02:35    -1102.041536        0.2907
BFGS:    6 09:02:35    -1102.044336        0.0865
BFGS:    7 09:02:35    -1102.044753        0.0640
BFGS:    8 09:02:35    -1102.044997        0.0545
BFGS:    9 09:02:35    -1102.045190        0.0401
      Step     Time          Energy         fmax
BFGS:    0 09:02:35    -1101.259714        9.4783
BFGS:    1 09:02:35    -1101.785196        3.2963
BFGS:    2 09:02:35    -1101.947484        1.3272
BFGS:    3 09:02:35    -1101.993016        0.7690
BFGS:    4 09:02:35    -1102.015729        0.8706
BFGS:    5 09:02:35    -1102.043306        0.2651
BFGS:    6 09:02:35    -1102.044582        0.0978
BF

BFGS:    4 09:02:37    -1102.040168        0.2745
BFGS:    5 09:02:37    -1102.044756        0.1278
BFGS:    6 09:02:37    -1102.045133        0.0417
      Step     Time          Energy         fmax
BFGS:    0 09:02:37    -1101.821760        3.1580
BFGS:    1 09:02:37    -1101.981538        1.0007
BFGS:    2 09:02:37    -1102.018686        0.5894
BFGS:    3 09:02:37    -1102.035875        0.3196
BFGS:    4 09:02:37    -1102.039536        0.2513
BFGS:    5 09:02:37    -1102.045054        0.0718
BFGS:    6 09:02:37    -1102.045174        0.0267
      Step     Time          Energy         fmax
BFGS:    0 09:02:38    -1100.854946        3.2278
BFGS:    1 09:02:38    -1101.276984        2.2801
BFGS:    2 09:02:38    -1101.721177        2.6724
BFGS:    3 09:02:38    -1101.980012        1.5708
BFGS:    4 09:02:38    -1102.023437        0.7861
BFGS:    5 09:02:38    -1102.042444        0.3187
BFGS:    6 09:02:38    -1102.043866        0.1163
BFGS:    7 09:02:38    -1102.044523        0.0797
BF

BFGS:    7 09:02:40    -1102.044142        0.0913
BFGS:    8 09:02:40    -1102.044827        0.0804
BFGS:    9 09:02:40    -1102.045168        0.0581
BFGS:   10 09:02:40    -1102.045258        0.0171
      Step     Time          Energy         fmax
BFGS:    0 09:02:40    -1100.812490        6.9701
BFGS:    1 09:02:40    -1101.705167        2.8528
BFGS:    2 09:02:40    -1101.873355        1.2948
BFGS:    3 09:02:40    -1101.959912        0.8829
BFGS:    4 09:02:40    -1102.015227        0.6368
BFGS:    5 09:02:40    -1102.037221        0.4383
BFGS:    6 09:02:40    -1102.044519        0.0987
BFGS:    7 09:02:40    -1102.044890        0.0505
BFGS:    8 09:02:40    -1102.045032        0.0384
      Step     Time          Energy         fmax
BFGS:    0 09:02:40    -1101.553550        3.6550
BFGS:    1 09:02:40    -1101.767527        1.8616
BFGS:    2 09:02:40    -1101.891590        1.5320
BFGS:    3 09:02:40    -1102.018955        0.7865
BFGS:    4 09:02:40    -1102.038604        0.2820
BF

...

BFGS:    2 09:05:41    -1101.720008        1.9498
BFGS:    3 09:05:41    -1101.997102        0.8973
BFGS:    4 09:05:41    -1102.024440        0.4864
BFGS:    5 09:05:41    -1102.041185        0.1990
BFGS:    6 09:05:41    -1102.042544        0.1498
BFGS:    7 09:05:41    -1102.044606        0.1279
BFGS:    8 09:05:41    -1102.045157        0.0598
BFGS:    9 09:05:41    -1102.045264        0.0107
      Step     Time          Energy         fmax
BFGS:    0 09:05:41    -1101.093320        3.9926
BFGS:    1 09:05:41    -1101.521455        2.5842
BFGS:    2 09:05:41    -1101.819707        2.1577
BFGS:    3 09:05:41    -1101.991891        1.1927
BFGS:    4 09:05:41    -1102.036533        0.4340
BFGS:    5 09:05:41    -1102.042939        0.2068
BFGS:    6 09:05:41    -1102.044564        0.0709
BFGS:    7 09:05:41    -1102.044785        0.0656
BFGS:    8 09:05:41    -1102.045177        0.0449
      Step     Time          Energy         fmax
BFGS:    0 09:05:41    -1101.881947        2.8917
BF

BFGS:    1 09:05:44    -1101.835135        1.3764
BFGS:    2 09:05:44    -1101.973455        1.3794
BFGS:    3 09:05:44    -1102.029100        0.7014
BFGS:    4 09:05:44    -1102.042639        0.1753
BFGS:    5 09:05:44    -1102.044157        0.1134
BFGS:    6 09:05:44    -1102.044772        0.0783
BFGS:    7 09:05:44    -1102.044994        0.0636
BFGS:    8 09:05:44    -1102.045213        0.0354
      Step     Time          Energy         fmax
BFGS:    0 09:05:44    -1101.814612        2.3358
BFGS:    1 09:05:44    -1101.917381        1.1677
BFGS:    2 09:05:44    -1101.979649        0.8565
BFGS:    3 09:05:44    -1102.028001        0.5542
BFGS:    4 09:05:44    -1102.038787        0.3577
BFGS:    5 09:05:44    -1102.044851        0.0803
BFGS:    6 09:05:44    -1102.045074        0.0389
      Step     Time          Energy         fmax
BFGS:    0 09:05:44    -1102.014801        0.9760
BFGS:    1 09:05:44    -1102.030440        0.4426
BFGS:    2 09:05:44    -1102.036489        0.3283
BF

BFGS:    4 09:05:46    -1102.010452        0.7685
BFGS:    5 09:05:46    -1102.031480        0.5021
BFGS:    6 09:05:46    -1102.042917        0.1346
BFGS:    7 09:05:46    -1102.043698        0.1160
BFGS:    8 09:05:46    -1102.045019        0.0681
BFGS:    9 09:05:46    -1102.045237        0.0277
      Step     Time          Energy         fmax
BFGS:    0 09:05:46    -1101.936589        2.8889
BFGS:    1 09:05:46    -1102.020852        0.9156
BFGS:    2 09:05:46    -1102.033163        0.3123
BFGS:    3 09:05:46    -1102.039833        0.2345
BFGS:    4 09:05:46    -1102.043255        0.1893
BFGS:    5 09:05:46    -1102.044817        0.0983
BFGS:    6 09:05:46    -1102.045199        0.0333
      Step     Time          Energy         fmax
BFGS:    0 09:05:46    -1100.725934        8.1636
BFGS:    1 09:05:46    -1101.425646        4.4099
BFGS:    2 09:05:46    -1101.703920        1.6956
BFGS:    3 09:05:46    -1101.888164        1.3676
BFGS:    4 09:05:46    -1101.973835        1.3262
BF

      Step     Time          Energy         fmax
BFGS:    0 09:05:49    -1101.909843        1.9744
BFGS:    1 09:05:49    -1101.986263        0.8077
BFGS:    2 09:05:49    -1102.012548        0.5683
BFGS:    3 09:05:49    -1102.034042        0.4571
BFGS:    4 09:05:49    -1102.041078        0.2429
BFGS:    5 09:05:49    -1102.044631        0.1169
BFGS:    6 09:05:49    -1102.045090        0.0417
      Step     Time          Energy         fmax
BFGS:    0 09:05:49    -1101.854376        1.9722
BFGS:    1 09:05:49    -1101.945361        1.0661
BFGS:    2 09:05:49    -1101.992540        0.8629
BFGS:    3 09:05:49    -1102.027119        0.4655
BFGS:    4 09:05:49    -1102.039536        0.2874
BFGS:    5 09:05:49    -1102.044408        0.1410
BFGS:    6 09:05:49    -1102.045032        0.0503
BFGS:    7 09:05:49    -1102.045140        0.0336
      Step     Time          Energy         fmax
BFGS:    0 09:05:49    -1101.388643        3.4052
BFGS:    1 09:05:49    -1101.683416        1.8467
BFG

BFGS:    2 09:05:51    -1101.990009        0.6548
BFGS:    3 09:05:51    -1102.034087        0.4504
BFGS:    4 09:05:51    -1102.041833        0.2905
BFGS:    5 09:05:51    -1102.044578        0.1203
BFGS:    6 09:05:51    -1102.045006        0.0507
BFGS:    7 09:05:51    -1102.045120        0.0309
      Step     Time          Energy         fmax
BFGS:    0 09:05:51    -1101.139318        5.7722
BFGS:    1 09:05:51    -1101.575180        3.8223
BFGS:    2 09:05:51    -1101.777547        1.4689
BFGS:    3 09:05:51    -1101.950096        1.2512
BFGS:    4 09:05:51    -1102.007628        1.1018
BFGS:    5 09:05:51    -1102.036902        0.4777
BFGS:    6 09:05:51    -1102.043723        0.1227
BFGS:    7 09:05:51    -1102.044328        0.0872
BFGS:    8 09:05:51    -1102.044806        0.0789
BFGS:    9 09:05:51    -1102.045154        0.0423
      Step     Time          Energy         fmax
BFGS:    0 09:05:51    -1101.523254        3.6480
BFGS:    1 09:05:51    -1101.802014        2.0870
BF

In [16]:
len(ch4_energies)

1376

In [17]:
rounded_ch4_energies = [round(i,5) for i in ch4_energies]
print(rounded_ch4_energies)

[-40.49941, -40.49941, -40.49941, -40.49942, -40.49942, -40.49942, -40.49941, -40.49941, -40.49942, -40.49941, -40.49941, -40.49941, -40.49941, -40.49941, -40.49941, -40.49942, -40.49942, -40.49942, -40.49942, -40.49941, -40.49941, -40.49941, -40.49941, -40.49941, -40.49942, -40.49942, -40.49941, -40.49941, -40.49942, -40.49941, -40.49941, -40.49942, -40.49941, -40.49942, -40.49941, -40.49942, -40.49941, -40.49941, -40.49941, -40.49942, -40.49941, -40.49941, -40.49941, -40.49941, -40.49941, -40.49942, -40.49942, -40.49941, -40.49942, -40.49941, -40.49941, -40.49941, -40.49942, -40.49942, -40.49941, -40.49941, -40.49941, -40.49941, -40.4994, -40.49942, -40.49941, -40.49942, -40.49941, -40.49942, -40.49941, -40.49941, -40.49942, -40.49942, -40.49941, -40.49942, -40.49942, -40.49941, -40.49942, -40.49941, -40.49942, -40.49941, -40.49942, -40.49942, -40.49942, -40.49942, -40.49941, -40.49941, -40.49941, -40.49941, -40.49941, -40.49942, -40.49941, -40.4994, -40.49941, -40.49941, -40.49941, 